## Extract Meta-Triples from Biothings SemMedDB API's Predication Data

### Overview

Reference: https://github.com/biothings/pending.api/issues/63#issuecomment-1100469563

A meta-triple is a unit of `(SUBJECT_SEMTYPE, PREDICATE, OBJECT_SEMTYPE)` values (see table below). E.g. `(dsyn, PROCESS_OF, humn)`, which can be roughly interpreted as "Disease or SYNdrome is a PROCESS OF HUMaN". Statistics on such meta-triples can help determine related x-bte annotations. Later we find that the entity types of subjects/objects are also helpful to x-bte developers, so two more fields, `SUBJECT_PREFIX` and `OBJECT_PREFIX`, are also included in the statistics. However we still call it a meta-triple.

The source file (`semmedVER43_2023_R_PREDICATION.116080_clean_pyarrow_snappy.parquet`) we use here is a cleaner and wangled version of the original SemMedDB predications (`semmedVER43_2023_R_PREDICATION.116080`), and it's stored in [Apache Parquet](https://parquet.apache.org/) format for better I/O performance. It can be found on server `su06` under directory `/data/pending/datasources/semmeddb/43/CACHE`.

The columns of the source file are listed below:

|Column Name     |Remark                                                        |
|----------------|--------------------------------------------------------------|
|`_ID`           | ID of the document parsed from this row                      |
|`PREDICATION_ID` | ID of the predication                                        |
|`PREDICATE`     | Predicate of the predication                                 |
|`PMID`          | PubMed ID of the predication                                 |
|`SUBJECT_CUI`   | Subject's CUI (either UMLS CUI or NCBIGene ID)               |
|`SUBJECT_PREFIX` | Indicator of Subject's CUI type (either `"umls"` or `"ncbigene"`)|
|`SUBJECT_NAME`  | Subject's name                                               |
|`SUBJECT_SEMTYPE` | Subject's semantic type (4-letter abbreviation)              |
|`SUBJECT_NOVELTY` | Subject's novelty score (alway 1 currently)                  |
|`OBJECT_CUI`    | Object's CUI (either UMLS CUI or NCBIGene ID)                |
|`OBJECT_PREFIX` | Indicator of Object's CUI type (either `"umls"` or `"ncbigene"`)|
|`OBJECT_NAME`   | Object's name                                                |
|`OBJECT_SEMTYPE` | Object's semantic type (4-letter abbreviation)               |
|`OBJECT_NOVELTY` | Object's novelty score (alway 1 currently)                   |

### Loading Data

Apache Arrow is an ideal in-memory transport layer for data that is being read or written with Parquet files. To load the parquet file, we need to install `pyarrow` package. E.g. with `pip`:

```bash
pip install pyarrow
```

Read more:

- [Installing PyArrow](https://arrow.apache.org/docs/python/install.html)
- [Reading and Writing the Apache Parquet Format](https://arrow.apache.org/docs/python/parquet.html)

In [ ]:
import pandas as pd

semmed_df = pd.read_parquet(path="semmedVER43_2023_R_PREDICATION.116080_clean_pyarrow_snappy.parquet", engine="pyarrow",
                            columns=["SUBJECT_PREFIX", "SUBJECT_SEMTYPE", "PREDICATE", "OBJECT_PREFIX", "OBJECT_SEMTYPE", "PMID"])

semmed_df.shape

In [ ]:
semmed_df.head()

### Meta-Triples vs No. Predications

Because each row is a predication, we can do `.value_counts()` for each combination of `SUBJECT_PREFIX`, `SUBJECT_SEMTYPE`, `PREDICATE`, `OBJECT_PREFIX`, and `OBJECT_SEMTYPE`, for the No. predications of each meta-triple.

In [ ]:
pred_stat = semmed_df.value_counts(subset=["SUBJECT_PREFIX", "SUBJECT_SEMTYPE", "PREDICATE", "OBJECT_PREFIX", "OBJECT_SEMTYPE"]).reset_index(name="PREDICATION_N")
pred_stat.shape

In [ ]:
# List the top 10 meta-triples
pred_stat.head(n=10)

In [ ]:
# List the top 10 meta-triples whose subject is a NCBIGene
pred_stat.loc[pred_stat["SUBJECT_PREFIX"].eq("ncbigene")].head(n=10)

In [ ]:
# Save the Result
pred_stat.to_csv("meta_triple_predication_stat.tsv", sep="\t", index=False)

### Meta-Triples vs No. Documents

Each semmeddb document has an ID made from 3 fields, `SUBJECT_CUI`, `PREDICATE`, and `OBJECT_CUI`. Therefore we group the input data frame by these 3 fields, and each group should contribute only 1 document to the stats. However on BTE ends, documents with `PMID` counts less than or equal 3 are often excluded due to their low significance. We also take the valid contribution of documents into account in making the statistics.

In [ ]:
g = semmed_df.groupby(["SUBJECT_CUI", "PREDICATE", "OBJECT_CUI"])

# Count the number of unique PMIDs for each group
doc_stat = g["PMID"].nunique().reset_index(name="PMID_N_UNIQUE")

# Each group should contribute only 1 document
doc_stat["DOC_N"] = 1

# If a group has less than or equal 3 PMIDs, it's valid contribution is 0
doc_stat["DOC_N_VALID"] = doc_stat["DOC_N"].where(doc_stat["PMID_N_UNIQUE"] > 3, other=0)

doc_stat.head()

Now join the document contribution stats to the orignal data frame. Note that a subject/object may have multiple semtypes. E.g. in [C0009325-INHIBITS-C0162574](https://biothings.transltr.io/semmeddb/association/C0009325-INHIBITS-C0162574), the subject has two semtypes, `aapp` and `phsu`, and it's mapped to two meta-triples, `(aapp, INHIBITS, bacs)` and `(phsu, NHIBITS, bacs)`. In such cases, all the meta-triples will receive the contribution of documents from the same ID.

In [ ]:
# TODO Can we use transform for all the 3 new columns, without merging? E.g. semmed_df["PMID_N_UNIQUE"] = g["PMID"].transform("nunique")
semmed_df = semmed_df.merge(doc_stat, how="inner", on=["SUBJECT_CUI", "PREDICATE", "OBJECT_CUI"])

In [ ]:
# Group by the meta-triples
g = semmed_df.groupby(["SUBJECT_PREFIX", "SUBJECT_SEMTYPE", "PREDICATE", "OBJECT_PREFIX", "OBJECT_SEMTYPE"])

"""
Here I encountered a bug in Pandas.GroupBy, whose root cause might be in the pyarrow implementation of GroupBy. 

If I call `g["DOC_N"].sum()`, it will exhaust the Cartesian product of all the values from the five columns, 
    "SUBJECT_PREFIX", "SUBJECT_SEMTYPE", "PREDICATE", "OBJECT_PREFIX", and "OBJECT_SEMTYPE", leading to numerous empty groups with sum of "DOC_N" equal 0,
    which is quite counter-intuitive because each group's DOC_N shoule be at least 1.
    
Therefore I take the following workaroud to make the aggregation manually.
"""
doc_stat2 = [[*index, data["DOC_N"].sum(), data["DOC_N_VALID"].sum()] for index, data in g]
doc_stat2 = pd.DataFrame(doc_stat2, columns=["SUBJECT_PREFIX", "SUBJECT_SEMTYPE", "PREDICATE", "OBJECT_PREFIX", "OBJECT_SEMTYPE", "DOC_N", "DOC_N_VALID"])
doc_stat2.sort_values("DOC_N_VALID", ascending=False, inplace=True)

In [ ]:
# Save the Result
doc_stat2.to_csv("meta_triple_document_stat.tsv", sep="\t", index=False)